In [1]:
import sys
sys.path.append('../')
sys.path.append('/Users/sdbykov/work/my_packages/nway')

In [2]:
from scripts.utils import data_path, set_mpl
from scripts.cross_match_scripts import cat2hpx, pandas_to_fits, my_scaler_forward, desi_reliable_magnitudes
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import astropy.io.fits as fits
import healpy as hp
from tensorflow import keras

import nway_python_api as nway
%matplotlib inline

matplotlib settings set


matplotlib settings set
This is NWAY Python API
nwaylib file /Users/sdbykov/work/my_packages/nway/nwaylib/__init__.py


# LH DESI catalog

All DESI objects within LH area + a little boundary


In [3]:
desi_cat_orig = pd.read_pickle(data_path+'desi_lh.gz_pkl', compression='gzip')

In [4]:
desi_cat_orig = desi_reliable_magnitudes(desi_cat_orig, s_n_threshold=3)
desi_cat_orig

/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:405: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:405: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:405: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:405: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/sdbykov/opt/miniconda3/envs/venv_hea/lib/python3.10/site-packages/pandas/core/arraylike.py:405: RuntimeWarning: invalid value encountered in log10
  result = getattr

,release,objid,brickid,ra,dec,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,type,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,ref_cat,ref_id,mjd_max,mjd_min,iso_max,iso_min,desi_id,all_mag_g,rel_mag_g,rel_dered_mag_g,rel_flux_corr_g,all_mag_r,rel_mag_r,rel_dered_mag_r,all_mag_z,rel_mag_z,rel_dered_mag_z,all_mag_w1,rel_mag_w1,rel_dered_mag_w1,vega_mag_w1,all_mag_w2,rel_mag_w2,rel_dered_mag_w2,vega_mag_w2,all_mag_w3,rel_mag_w3,rel_dered_mag_w3,vega_mag_w3,all_mag_w4,rel_mag_w4,rel_dered_mag_w4,vega_mag_w4,rel_flux_corr_r,rel_flux_corr_z,rel_dered_g_r,rel_dered_g_z,rel_dered_r_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4
0,9011,3023,623750,161.795663,62.309075,0.278886,0.729804,0.925065,-0.124014,0.842693,7.723400,-409.541560,628.14530,241.19875,67.662970,4.827379,1.161493,0.001375,0.000017,23.855670,22.821260,22.572979,NaN,22.684746,20.280249,NaN,6.989662,11.334282,7.609353,-0.272475,0.908192,0.286416,-1.682871,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58245.183300,57821.352569,2018-05-07 04:23:57.150,2017-03-09 08:27:42,9011_623750_3023,23.886433,23.886433,23.855670,1.653265e-15,22.841984,22.841984,22.821260,22.584569,22.584569,22.572979,NaN,NaN,NaN,NaN,22.685827,NaN,NaN,NaN,20.280479,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.600611e-15,1.552877e-15,1.034410,1.282691,0.248281,NaN,NaN,NaN,NaN,NaN,NaN
1,9011,2993,623750,161.789851,62.306325,0.832196,1.570136,3.506991,17.061989,15.794795,-11.218180,293.149100,381.48358,190.84026,46.138702,3.873229,1.025061,0.001371,0.000017,22.668740,21.989480,21.126097,19.418169,19.502636,NaN,16.332191,16.254122,21.690641,23.821402,33.578884,15.991490,-0.415374,1.201052,EXP,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58245.183300,57821.352569,2018-05-07 04:23:57.150,2017-03-09 08:27:42,9011_623750_2993,22.699436,22.699436,22.668740,4.933038e-15,22.010157,22.010157,21.989480,21.137663,21.137663,21.126097,19.419926,19.419926,19.418169,16.720926,19.503715,19.503715,19.502636,16.164715,NaN,NaN,NaN,NaN,16.332279,NaN,NaN,NaN,5.594837e-15,5.886943e-15,0.679260,1.542643,0.863383,1.707928,2.486844,-0.084467,NaN,NaN,NaN
2,9011,3026,623750,161.796264,62.305792,0.264495,0.484687,0.485263,-0.030457,-0.994314,-2.437728,215.115220,423.51940,211.87927,52.782760,4.773809,1.144707,0.001378,0.000017,23.913218,23.265640,23.273476,NaN,NaN,NaN,16.668236,5.443207,7.055143,3.525519,-0.066547,-1.063826,-0.090477,0.881277,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58245.183300,57821.352569,2018-05-07 04:23:57.150,2017-03-09 08:27:42,9011_623750_3026,23.943956,23.943956,23.913218,1.567918e-15,23.286347,23.286347,23.265640,23.285057,23.285057,23.273476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.668322,NaN,NaN,NaN,1.727124e-15,8.145885e-16,0.647578,0.639742,-0.007836,NaN,NaN,NaN,NaN,NaN,NaN
3,9011,3215,623750,161.824788,62.288017,0.606415,0.892139,1.129547,0.917347,1.244101,-56.614063,-77.085710,482.92303,192.82315,42.934322,4.735627,1.142404,0.001443,0.000018,23.012274,22.603170,22.356134,22.591902,22.261778,NaN,NaN,13.326289,12.388316,7.401279,1.996284,1.329736,-2.150374,-0.323718,EXP,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58245.183300,57821.352569,2018-05-07 04:23:57.150,2017-03-09 08:27:42,9011_623750_3215,23.043075,23.043075,23.012274,3.595016e-15,22.623919,22.623919,22.603170,22.367739,22.367739,22.356134,22.593666,NaN,NaN,NaN,22.262861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.179153e-15,1.896160e-15,0.409104,0.656140,0.247036,NaN,NaN,NaN,NaN,NaN,NaN
4,9011,3213,623750,161.824698,62.283593,-0.009455,0.147133,1.737552,10.260174,8.766667,-26.475445,194.974380,609.35693,232.28662,37.336655,4.246073,1.077639,0.001466,0.000018,NaN,24.559986,21.888557,19.970350,20.141830,NaN,16.774970,-0.233407,2.242455,10.617088,21.142117,9.100624,-1.013735,0.815675,REX,0.0,0.0,0.0,0.0,0.0,0.0,Na

countring missing data

In [5]:
print('fraction of missed measurements per filter, in per cent:')
desi_cat_orig[['rel_dered_mag_g',	'rel_dered_mag_r',	'rel_dered_mag_z',	'rel_dered_mag_w1',	'rel_dered_mag_w2',	'rel_dered_mag_w3',	'rel_dered_mag_w4']].agg(lambda x: np.round(100*np.mean(np.
isnan(x)),1))

fraction of missed measurements per filter, in per cent:


rel_dered_mag_g     17.6
rel_dered_mag_r     11.9
rel_dered_mag_z      6.6
rel_dered_mag_w1    35.1
rel_dered_mag_w2    59.5
rel_dered_mag_w3    94.8
rel_dered_mag_w4    90.2
dtype: float64

In [6]:
print('grz all nan:', len(desi_cat_orig.query('rel_dered_mag_g.isna() & rel_dered_mag_r.isna() & rel_dered_mag_z.isna()'))/len(desi_cat_orig))

#print('gr is not nan, z is nan :', len(desi_cat_orig.query('~rel_dered_mag_g.isna() & ~rel_dered_mag_r.isna() & rel_dered_mag_z.isna()'))/len(desi_cat_orig))
#print('rz is not nan, g is nan:', len(desi_cat_orig.query('~rel_dered_mag_r.isna() & ~rel_dered_mag_z.isna() & rel_dered_mag_g.isna()'))/len(desi_cat_orig))
#print('gz is not nan, r is nan:', len(desi_cat_orig.query('~rel_dered_mag_g.isna() & ~rel_dered_mag_z.isna() & rel_dered_mag_r.isna()'))/len(desi_cat_orig))

print('all grz is not nan:', len(desi_cat_orig.query('~rel_dered_mag_g.isna() & ~rel_dered_mag_r.isna() & ~rel_dered_mag_z.isna()'))/len(desi_cat_orig))

print('grz is not nan, w1 is not nan, w2 is nan:', len(desi_cat_orig.query('~rel_dered_mag_g.isna() & ~rel_dered_mag_r.isna() & ~rel_dered_mag_z.isna() & ~rel_dered_mag_w1.isna() & rel_dered_mag_w2.isna()'))/len(desi_cat_orig))

print('grz is not nan, w2 is not nan, w1 is nan:', len(desi_cat_orig.query('~rel_dered_mag_g.isna() & ~rel_dered_mag_r.isna() & ~rel_dered_mag_z.isna() & rel_dered_mag_w1.isna() & ~rel_dered_mag_w2.isna()'))/len(desi_cat_orig))


print('grz is not nan, w1 and w2 is not nan', len(desi_cat_orig.query('~rel_dered_mag_g.isna() & ~rel_dered_mag_r.isna() & ~rel_dered_mag_z.isna() & ~rel_dered_mag_w1.isna() & ~rel_dered_mag_w2.isna()'))/len(desi_cat_orig))

grz all nan: 0.0002840516767318263
all grz is not nan: 0.7246063176069866
grz is not nan, w1 is not nan, w2 is nan: 0.19196973092408998
grz is not nan, w2 is not nan, w1 is nan: 0.008405366137236239
grz is not nan, w1 and w2 is not nan 0.29915355081134587


filtering the catalog

In [7]:
desi_cat = desi_cat_orig.sort_values(by=['flux_g'], ascending=[False])
desi_cat = desi_cat.drop_duplicates(subset=['ra', 'dec'], keep='first')



columns_to_retain = ['desi_id', 'ra', 'dec', 'rel_dered_mag_g','rel_dered_mag_r','rel_dered_mag_z','rel_dered_mag_w1','rel_dered_mag_w2', 'rel_dered_mag_w3', 'rel_dered_mag_w4', 'rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z','rel_dered_z_w1', 'rel_dered_r_w2', 'rel_dered_w1_w2', 'rel_dered_z_w3', 'rel_dered_r_w4', 'rel_dered_w3_w4']

desi_cat = desi_cat[columns_to_retain]


desi_cat.reset_index(inplace=True, drop = True)
desi_cat.sample(5)

,desi_id,ra,dec,rel_dered_mag_g,rel_dered_mag_r,rel_dered_mag_z,rel_dered_mag_w1,rel_dered_mag_w2,rel_dered_mag_w3,rel_dered_mag_w4,rel_dered_g_r,rel_dered_r_z,rel_dered_g_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4
1560961,9011_619606_3262,163.120549,60.677668,24.101261,22.717249,21.934969,20.666983,21.075546,NaN,NaN,1.384012,0.782280,2.166292,1.267986,1.641703,-0.408563,NaN,NaN,NaN
1758953,9011_609933_1796,155.204585,57.456645,24.287420,24.020683,22.874157,21.644688,21.236713,NaN,NaN,0.266737,1.146526,1.413263,1.229469,2.783970,0.407975,NaN,NaN,NaN
737851,9011_605190_1163,163.188338,55.955640,23.252888,22.862944,NaN,21.112858,NaN,17.294506,NaN,0.389944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1713493,9011_601071_1133,159.410977,54.812417,24.243977,23.096025,22.612862,21.349846,NaN,NaN,NaN,1.147952,0.483163,1.631115,1.263016,NaN,NaN,NaN,NaN,NaN
2164850,9011_610724_3603,162.724494,57.719655,NaN,NaN,20.840641,21.263493,20.867520,NaN,NaN,NaN,NaN,NaN,-0.422852,NaN,0.395973,NaN,NaN,NaN


## Dealing with missing data -> assigning prior type

In [8]:
desi_cat['prior_type'] = 'None'


mask = ~(desi_cat.rel_dered_mag_g.isna()) & ~(desi_cat.rel_dered_mag_r.isna()) & ~(desi_cat.rel_dered_mag_z.isna())
desi_cat.loc[mask, 'prior_type'] = 'grz'
mask = mask & ~(desi_cat.rel_dered_mag_w1.isna())
desi_cat.loc[mask, 'prior_type'] = 'grzw1'
mask = mask & ~(desi_cat.rel_dered_mag_w2.isna()) 
desi_cat.loc[mask, 'prior_type'] = 'grzw1w2'

print(desi_cat.prior_type.value_counts())
print(desi_cat.prior_type.value_counts(normalize=1))

grzw1w2    723525
None       666060
grz        564696
grzw1      464293
Name: prior_type, dtype: int64
grzw1w2    0.299154
None       0.275394
grz        0.233483
grzw1      0.191970
Name: prior_type, dtype: float64


In [9]:
desi_cat['prior_type'] = 'None'

mask = ~(desi_cat.rel_dered_mag_g.isna()) & ~(desi_cat.rel_dered_mag_r.isna()) & ~(desi_cat.rel_dered_mag_z.isna())
desi_cat.loc[mask, 'prior_type'] = 'grz'
mask = mask & ~(desi_cat.rel_dered_mag_w1.isna())
desi_cat.loc[mask, 'prior_type'] = 'grzw1'
mask = mask & ~(desi_cat.rel_dered_mag_w2.isna()) 
desi_cat.loc[mask, 'prior_type'] = 'grzw1w2'

#desi_cat_scaled_imputed.loc[desi_cat_scaled_imputed.prior_type==prior, features_cols] = -99



In [10]:
print(desi_cat.prior_type.value_counts())
print(desi_cat.prior_type.value_counts(normalize=1))

grzw1w2    723525
None       666060
grz        564696
grzw1      464293
Name: prior_type, dtype: int64
grzw1w2    0.299154
None       0.275394
grz        0.233483
grzw1      0.191970
Name: prior_type, dtype: float64


## Adding prior columns

In [11]:
desi_cat_scaled = my_scaler_forward(desi_cat) #scale the data for KERAS NN applying

rel_dered_mag_g scaled by 1/35
rel_dered_mag_r scaled by 1/35
rel_dered_mag_z scaled by 1/35
rel_dered_mag_w1 scaled by 1/35
rel_dered_mag_w2 scaled by 1/35
rel_dered_mag_w3 scaled by 1/35
rel_dered_mag_w4 scaled by 1/35
rel_dered_g_r scaled by 1/10
rel_dered_r_z scaled by 1/10
rel_dered_g_z scaled by 1/10
rel_dered_z_w1 scaled by 1/10
rel_dered_r_w2 scaled by 1/10
rel_dered_w1_w2 scaled by 1/10
rel_dered_z_w3 scaled by 1/10
rel_dered_r_w4 scaled by 1/10
rel_dered_w3_w4 scaled by 1/10


In [12]:
#print(pd.DataFrame(desi_cat_scaled.describe().loc[['mean', 'std',  'max', 'min']]).T)
desi_cat_scaled_imputed= desi_cat_scaled.fillna(value = -99)


In [13]:
for prior in ['grzw1w2', 'grzw1', 'grz']:
    nnmag_name = 'nnmag_'+prior
    desi_cat_scaled_imputed[nnmag_name] = -99
    nnmag_name = 'nnmag_'+prior+'_orig'
    desi_cat_scaled_imputed[nnmag_name] = -99




In [ ]:
model_grz = keras.models.load_model('/Users/sdbykov/work/lockman_hole/1_desi-photo-prior/model/nnmag_grz.keras_nn')
model_grzw1 = keras.models.load_model('/Users/sdbykov/work/lockman_hole/1_desi-photo-prior/model/nnmag_grzw1.keras_nn')
model_grzw1w2 = keras.models.load_model('/Users/sdbykov/work/lockman_hole/1_desi-photo-prior/model/nnmag_grzw1w2.keras_nn')


def make_predictions(df, prior_type):

    features_cols_grzw1w2 =  ['rel_dered_mag_g', 'rel_dered_mag_r', 'rel_dered_mag_z', 'rel_dered_mag_w1', 'rel_dered_mag_w2', 'rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z', 'rel_dered_z_w1', 'rel_dered_r_w2', 'rel_dered_w1_w2']
    features_cols_grzw1 =  ['rel_dered_mag_g', 'rel_dered_mag_r', 'rel_dered_mag_z', 'rel_dered_mag_w1', 'rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z', 'rel_dered_z_w1']
    features_cols_grz =  ['rel_dered_mag_g', 'rel_dered_mag_r', 'rel_dered_mag_z', 'rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z']

    print('Calculating predictions for prior_type: '+prior_type)
    if prior_type == 'grzw1w2':
        features_cols = features_cols_grzw1w2
        model = model_grzw1w2
    elif prior_type == 'grzw1':
        features_cols = features_cols_grzw1
        model = model_grzw1
    elif prior_type == 'grz':
        features_cols = features_cols_grz
        model = model_grz
    else:
        raise ValueError('prior_type type not recognized')

        
    mask = df.prior_type==prior_type
    df_tmp = df[mask][features_cols]


    nnmag_name = 'nnmag_'+prior_type
    X_predict = df_tmp.values
    y_predict = model.predict(X_predict)

    df.loc[mask, nnmag_name] = y_predict
    df.loc[mask, nnmag_name+'_orig'] = y_predict

    if prior_type=='grzw1' or prior_type=='grzw1w2':
        print('--Calculating sub-predictions for prior_type: '+'grz')

        X_predict =  df[mask][features_cols_grz].values
        y_predict = model_grz.predict(X_predict)
        df.loc[mask, 'nnmag_grz_orig'] = y_predict


    if prior_type=='grzw1w2':
        print('--Calculating sub-predictions for prior_type: '+'grzw1')
        X_predict =  df[mask][features_cols_grzw1].values
        y_predict = model_grzw1.predict(X_predict)
        df.loc[mask, 'nnmag_grzw1_orig'] = y_predict

    return df

for prior_type in ['grzw1w2', 'grzw1', 'grz']:
    desi_cat_scaled_imputed = make_predictions(desi_cat_scaled_imputed, prior_type)
   

In [15]:
desi_cat_scaled_imputed.query("nnmag_grz_orig!=-99")

,desi_id,ra,dec,rel_dered_mag_g,rel_dered_mag_r,rel_dered_mag_z,rel_dered_mag_w1,rel_dered_mag_w2,rel_dered_mag_w3,rel_dered_mag_w4,rel_dered_g_r,rel_dered_r_z,rel_dered_g_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,prior_type,nnmag_grzw1w2,nnmag_grzw1w2_orig,nnmag_grzw1,nnmag_grzw1_orig,nnmag_grz,nnmag_grz_orig
20,9011_608391_3304,166.193423,56.994008,0.235522,0.174716,0.417170,0.348593,0.367892,0.405444,0.416527,0.212819,-0.848586,-0.635767,0.240018,-0.676117,-0.067548,0.041039,-0.846337,-0.038790,grzw1w2,0.683062,0.683062,-99.000000,0.734866,-99.000000,0.796039
31,9011_611477_2811,155.048961,57.897765,0.248089,0.370034,0.388594,0.322769,0.341074,0.390173,0.416863,-0.426805,-0.064961,-0.491766,0.230386,0.101359,-0.064066,-0.005527,-0.163904,-0.093417,grzw1w2,0.951225,0.951225,-99.000000,0.969611,-99.000000,0.926494
35,9011_605987_1393,159.127645,56.156413,0.253675,0.288068,0.349741,0.322860,0.342291,0.390584,0.413188,-0.120375,-0.215855,-0.336230,0.094082,-0.189779,-0.068006,-0.142950,-0.437920,-0.079115,grzw1w2,0.911177,0.911177,-99.000000,0.966600,-99.000000,0.824262
39,9011_603553_3101,159.774676,55.512338,0.259401,0.323637,0.426948,0.283841,0.295961,0.347407,0.381680,-0.224824,-0.361589,-0.586413,0.500874,0.096866,-0.042419,0.278393,-0.203152,-0.119957,grzw1w2,0.951231,0.951231,-99.000000,0.969274,-99.000000,0.419905
46,9011_608371_1748,156.891014,56.878860,0.264951,0.322180,0.427598,0.305930,0.323325,0.373075,0.409076,-0.200300,-0.368964,-0.569264,0.425838,-0.004009,-0.060883,0.190830,-0.304138,-0.126003,grzw1w2,0.951199,0.951199,-99.000000,0.969334,-99.000000,0.378270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2166732,9011_602739_3367,163.273652,55.206833,0.718991,0.674090,0.662804,-99.000000,-99.000000,-99.000000,-99.000000,0.157155,0.039499,0.196654,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,grz,-99.000000,-99.000000,-99.000000,-99.000000,0.032073,0.032073
2170164,9011_602740_3460,163.722208,55.210597,0.719333,0.693232,0.642976,0.582317,0.592495,-99.000000,-99.000000,0.091354,0.175897,0.267250,0.212306,0.352580,-0.035623,-99.000000,-99.000000,-99.000000,grzw1w2,0.130816,0.130816,-99.000000,0.097728,-99.000000,0.163845
2173125,9011_602740_2087,163.599114,55.220587,0.719602,0.662111,0.620208,-99.000000,-99.000000,-99.000000,-99.000000,0.201220,0.146661,0.347880,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,grz,-99.000000,-99.000000,-99.000000,-99.000000,0.154108,0.154108
2181308,9011_601078_4008,162.729909,54.805880,0.720728,0.675197,0.633285,0.611307,0.608974,-99.000000,-99.000000,0.159358,0.146691,0.306049,0.076925,0.231779,0.008163,-99.000000,-99.000000,-99.000000,grzw1w2,0.064276,0.064276,-99.000000,0.024005,-99.000000,0.202521


In [16]:
for prior in ['grzw1w2', 'grzw1', 'grz']:


    features_cols_grzw1w2 =  ['rel_dered_mag_g', 'rel_dered_mag_r', 'rel_dered_mag_z', 'rel_dered_mag_w1', 'rel_dered_mag_w2', 'rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z', 'rel_dered_z_w1', 'rel_dered_r_w2', 'rel_dered_w1_w2']
    features_cols_grzw1 =  ['rel_dered_mag_g', 'rel_dered_mag_r', 'rel_dered_mag_z', 'rel_dered_mag_w1', 'rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z', 'rel_dered_z_w1']
    features_cols_grz =  ['rel_dered_mag_g', 'rel_dered_mag_r', 'rel_dered_mag_z', 'rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z']

    print('Removing original  features columns for prior: '+prior)
    if prior == 'grzw1w2':
        features_cols = features_cols_grzw1w2
    elif prior == 'grzw1':
        features_cols = features_cols_grzw1
    elif prior == 'grz':
        features_cols = features_cols_grz
    else:
        raise ValueError('prior type not recognized')


    mask = desi_cat_scaled_imputed.prior_type==prior
    desi_cat_scaled_imputed.loc[mask, features_cols] = -99


Removing original  features columns for prior: grzw1w2
Removing original  features columns for prior: grzw1
Removing original  features columns for prior: grz


In [17]:
photo_columns = ['desi_id', 'rel_dered_mag_g','rel_dered_mag_r','rel_dered_mag_z','rel_dered_mag_w1','rel_dered_mag_w2', 'rel_dered_mag_w3', 'rel_dered_mag_w4', 'rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z','rel_dered_z_w1', 'rel_dered_r_w2', 'rel_dered_w1_w2', 'rel_dered_z_w3', 'rel_dered_r_w4', 'rel_dered_w3_w4']

desi_cat_scaled_imputed = desi_cat_scaled_imputed.merge(desi_cat[photo_columns], on='desi_id', how='left', suffixes=('', '_orig'))

desi_cat_scaled_imputed.sample(10)

,desi_id,ra,dec,rel_dered_mag_g,rel_dered_mag_r,rel_dered_mag_z,rel_dered_mag_w1,rel_dered_mag_w2,rel_dered_mag_w3,rel_dered_mag_w4,rel_dered_g_r,rel_dered_r_z,rel_dered_g_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,prior_type,nnmag_grzw1w2,nnmag_grzw1w2_orig,nnmag_grzw1,nnmag_grzw1_orig,nnmag_grz,nnmag_grz_orig,rel_dered_mag_g_orig,rel_dered_mag_r_orig,rel_dered_mag_z_orig,rel_dered_mag_w1_orig,rel_dered_mag_w2_orig,rel_dered_mag_w3_orig,rel_dered_mag_w4_orig,rel_dered_g_r_orig,rel_dered_r_z_orig,rel_dered_g_z_orig,rel_dered_z_w1_orig,rel_dered_r_w2_orig,rel_dered_w1_w2_orig,rel_dered_z_w3_orig,rel_dered_r_w4_orig,rel_dered_w3_w4_orig
2125822,9011_611492_176,161.690064,58.104149,-99.000000,-99.000000,0.662106,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.0,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.0,None,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,NaN,NaN,23.173717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2032612,9011_609167_3307,161.974855,57.154067,-99.000000,-99.000000,0.617211,0.555685,0.568061,-99.000000,-99.000000,-99.000000,-99.000000,-99.0,0.215343,-99.000000,-0.043317,-99.000000,-99.000000,-99.0,None,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,NaN,NaN,21.602396,19.448963,19.882130,NaN,NaN,NaN,NaN,NaN,2.153433,NaN,-0.433167,NaN,NaN,NaN
1053362,9011_608369_415,155.845260,57.006466,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.0,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.0,grz,-99.000000,-99.000000,-99.000000,-99.000000,0.210723,0.210723,23.617016,23.148989,22.480690,NaN,NaN,NaN,NaN,0.468027,0.668299,1.136326,NaN,NaN,NaN,NaN,NaN,NaN
2065147,9011_609174_2816,165.122839,57.194308,-99.000000,0.653755,0.618892,0.583139,0.601823,-99.000000,-99.000000,-99.000000,0.122021,-99.0,0.125138,0.181763,-0.065396,-99.000000,-99.000000,-99.0,None,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,NaN,22.881440,21.661230,20.409851,21.063808,NaN,NaN,NaN,1.220210,NaN,1.251379,1.817632,-0.653957,NaN,NaN,NaN
594202,9011_611481_3082,156.949647,57.920469,0.657417,0.637800,-99.000000,0.598206,-99.000000,-99.000000,-99.000000,0.068659,-99.000000,-99.0,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.0,None,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,23.009602,22.323008,NaN,20.937212,NaN,NaN,NaN,0.686594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227114,9011_612268_3635,167.685393,58.261766,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.0,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.0,grzw1w2,0.011236,0.011236,-99.000000,0.010658,-99.000000,0.368492,21.826820,20.510954,19.678000,20.464886,20.850878,NaN,NaN,1.315866,0.832954,2.148820,-0.786886,-0.339924,-0.385992,NaN,NaN,NaN
219202,9011_615252_1449,157.838417,59.297660,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.0,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.0,grzw1w2,0.008408,0.008408,-99.000000,0.010309,-99.000000,0.374454,21.789204,20.448906,19.566322,20.433504,21.131052,NaN,NaN,1.340298,0.882584,2.222882,-0.867182,-0.682146,-0.697548,NaN,NaN,NaN
2210717,9011_618175_3185,160.353053,60.158008,-99.000000,0.667663,0.613140,0.563845,0.573148,-99.000000,0.429379,-99.000000,0.190831,-99.0,0.172532,0.330802,-0.032561,-99.000000,0.833994,-99.0,None,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,NaN,23.368217,21.459904,19.734585,20.060196,NaN,15.028278,NaN,1.908313,NaN,1.725319,3.308021,-0.325611,NaN,8.339939,NaN
1558543,9011_617455_3642,161.094458,60.084938,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,0.500891,-99.000000,-99.000000,-99.000000,-99.0,-99.000000,-99.000000,-99.000000,0.330951,-99.000000,-99.0,grzw1,-99.000000,-99.000000,0.009983,0.009983,-99.000000,0.284920,24.091858,2

In [18]:
# #testing ideas-  indicator variable whether WISE w1 and w2 magnitudes are available or not.
# desi_cat_scaled_imputed['rel_dered_mag_w1_orig_is_missing'] = desi_cat_scaled_imputed['rel_dered_mag_w1_orig']==-99
# desi_cat_scaled_imputed['rel_dered_mag_w2_orig_is_missing'] = desi_cat_scaled_imputed['rel_dered_mag_w2_orig']==-99


In [19]:
desi_cat_scaled_imputed.groupby('prior_type')[['nnmag_grzw1w2', 'nnmag_grzw1', 'nnmag_grz']].agg(['max'])


,nnmag_grzw1w2,nnmag_grzw1,nnmag_grz
,max,max,max
prior_type,,,
None,-99.00000,-99.000000,-99.000000
grz,-99.00000,-99.000000,0.998955
grzw1,-99.00000,0.969212,-99.000000
grzw1w2,0.95125,-99.000000,-99.000000


In [20]:
desi_cat_scaled_imputed.groupby('prior_type')[['nnmag_grzw1w2_orig', 'nnmag_grzw1_orig', 'nnmag_grz_orig']].agg(['max'])


,nnmag_grzw1w2_orig,nnmag_grzw1_orig,nnmag_grz_orig
,max,max,max
prior_type,,,
None,-99.00000,-99.000000,-99.000000
grz,-99.00000,-99.000000,0.998955
grzw1,-99.00000,0.969212,0.998301
grzw1w2,0.95125,0.969611,0.999394


## area and plotting

In [21]:
%matplotlib inline
#desi_hp_map = cat2hpx(desi_cat_scaled_imputed['ra'], desi_cat_scaled_imputed['dec'], nside=1024, radec=True)

try:
    hp.gnomview(desi_hp_map, rot = [150.23,52.19], xsize = 400)
    DESI_SKYAREA = 4*np.pi*(np.rad2deg(1))**2 * np.sum(desi_hp_map>=1)/len(desi_hp_map)
except:
    DESI_SKYAREA = 41.728630922842704
print(DESI_SKYAREA, 'sq. deg.')

desi_sky_dens = len(desi_cat_scaled_imputed)/DESI_SKYAREA
print('DESI sky density:', desi_sky_dens, 'per sq deg')
print('DESI sky density:', desi_sky_dens/(3600), 'per sq. arcmin')
print('DESI sky density:', desi_sky_dens/(3600**2), 'per sq. arcsec')
print(' expected number of DESI sources within 1 arcsec radius', np.pi*1**2 * desi_sky_dens/(3600**2))
print(' expected number of DESI sources within 5 arcsec radius', np.pi*5**2 * desi_sky_dens/(3600**2))
print(' expected number of DESI sources within 10 arcsec radius', np.pi*10**2 * desi_sky_dens/(3600**2))



41.728630922842704 sq. deg.
DESI sky density: 57959.58186291816 per sq deg
DESI sky density: 16.0998838508106 per sq. arcmin
DESI sky density: 0.0044721899585585 per sq. arcsec
 expected number of DESI sources within 1 arcsec radius 0.014049799119265424
 expected number of DESI sources within 5 arcsec radius 0.3512449779816356
 expected number of DESI sources within 10 arcsec radius 1.4049799119265425


In [22]:
pandas_to_fits(dataframe = desi_cat_scaled_imputed, 
                filename = 'desi_lh.fits',
                table_header_name = 'DESI',
                sky_area_deg2 = DESI_SKYAREA)

In [23]:
desi_cat_scaled_imputed.query("prior_type == 'grzw1w2'")[['nnmag_grz_orig','nnmag_grzw1_orig', 'nnmag_grzw1w2_orig']].corr()

,nnmag_grz_orig,nnmag_grzw1_orig,nnmag_grzw1w2_orig
nnmag_grz_orig,1.000000,0.264629,0.114925
nnmag_grzw1_orig,0.264629,1.000000,0.741022
nnmag_grzw1w2_orig,0.114925,0.741022,1.000000


In [24]:
desi_cat_scaled_imputed.query("prior_type == 'grzw1w2'").iloc[0]

desi_id                  9011_608391_3304
ra                             166.193423
dec                             56.994008
rel_dered_mag_g                     -99.0
rel_dered_mag_r                     -99.0
rel_dered_mag_z                     -99.0
rel_dered_mag_w1                    -99.0
rel_dered_mag_w2                    -99.0
rel_dered_mag_w3                 0.405444
rel_dered_mag_w4                 0.416527
rel_dered_g_r                       -99.0
rel_dered_r_z                       -99.0
rel_dered_g_z                       -99.0
rel_dered_z_w1                      -99.0
rel_dered_r_w2                      -99.0
rel_dered_w1_w2                     -99.0
rel_dered_z_w3                   0.041039
rel_dered_r_w4                  -0.846337
rel_dered_w3_w4                  -0.03879
prior_type                        grzw1w2
nnmag_grzw1w2                    0.683062
nnmag_grzw1w2_orig               0.683062
nnmag_grzw1                         -99.0
nnmag_grzw1_orig                 0